In [ ]:
›import geopandas as gpd
import pandas as pd
import json
from shapely.geometry import Polygon, LineString
import numpy as np
import datetime
import os

# Dependencies
%pip install folium>=0.12 matplotlib mapclassify

We are loading vessel route information which contains the speed and each Geo location points in their route.

then we load Geo locations with the NDWI score.

In [ ]:
gdf_vessels_ndwi_scored = gpd.read_file('/content/mock_vessels_conditional.geojson')
gdf_ndwi_pixels = gpd.read_file('/content/ndwi_pixels_water.geojson')

print("Loaded gdf_vessels_ndwi_scored (head):")
display(gdf_vessels_ndwi_scored.head())

print("\nLoaded gdf_ndwi_pixels (head):")
display(gdf_ndwi_pixels.head())

Loaded gdf_vessels_ndwi_scored (head):


,id,vessel_type,speed_knots,timestamp,geometry
0,1,research_vessel_high_NDWI,3.0,2025-11-08 10:00:21.394,"LINESTRING (13.25385 52.571, 13.25007 52.56782..."
1,2,eco_tour_medium_NDWI,2.5,2025-11-08 16:00:21.395,"LINESTRING (13.28511 52.56726, 13.25254 52.578..."



Loaded gdf_ndwi_pixels (head):


,NDWI,geometry
0,-0.692628,POINT (13.20572 52.59718)
1,-0.618733,POINT (13.20587 52.59718)
2,-0.476723,POINT (13.20601 52.59718)
3,-0.476723,POINT (13.20616 52.59718)
4,-0.266003,POINT (13.2063 52.59718)


For each vessel route, spatially join it with the ndwi_pixels_water.geojson to identify all intersecting NDWI pixels and their corresponding NDWI scores. Then, calculate the average NDWI value for the pixels that fall along each vessel's path.

In [ ]:
print("Performing spatial join between vessel routes and NDWI pixels...")
# Spatial join: find all NDWI pixels that intersect each vessel route
# Using 'intersects' predicate to find pixels that touch or cross the LineString
vessels_with_ndwi_pixels = gpd.sjoin(gdf_vessels_ndwi_scored, gdf_ndwi_pixels, how="left", predicate="intersects")

print(f"Found {len(vessels_with_ndwi_pixels)} intersections between vessels and NDWI pixels.")

# Group by vessel 'id' and calculate the mean NDWI value for each vessel
print("Calculating average NDWI score per vessel...")

avg_ndwi_per_vessel = vessels_with_ndwi_pixels.groupby('id')['NDWI'].mean().reset_index()
avg_ndwi_per_vessel = avg_ndwi_per_vessel.rename(columns={'NDWI': 'avg_ndwi_score'})

# Merge these average NDWI values back into the original gdf_vessels_ndwi_scored GeoDataFrame
print("Merging average NDWI scores back to vessel GeoDataFrame...")
gdf_vessels_ndwi_scored = gdf_vessels_ndwi_scored.merge(avg_ndwi_per_vessel, on='id', how='left')

print("gdf_vessels_ndwi_scored with 'avg_ndwi_score' column (head):")
display(gdf_vessels_ndwi_scored.head())

Performing spatial join between vessel routes and NDWI pixels...
Found 21 intersections between vessels and NDWI pixels.
Calculating average NDWI score per vessel...
Merging average NDWI scores back to vessel GeoDataFrame...
gdf_vessels_ndwi_scored with 'avg_ndwi_score' column (head):


,id,vessel_type,speed_knots,timestamp,geometry,avg_ndwi_score
0,1,research_vessel_high_NDWI,3.0,2025-11-08 10:00:21.394,"LINESTRING (13.25385 52.571, 13.25007 52.56782...",0.504131
1,2,eco_tour_medium_NDWI,2.5,2025-11-08 16:00:21.395,"LINESTRING (13.28511 52.56726, 13.25254 52.578...",0.284545


a function to calculate the overall eco-score for each vessel, combining average NDWI value and vessel speed, and then apply this function to the GeoDataFrame.

In [ ]:
def calculate_eco_score_ndwi_speed(row):
    """
    Calculates an eco-score combining average NDWI value and vessel speed.
    Higher NDWI is better, lower speed is better.
    The score is normalized to be between 0 and 100.
    """
    # Handle potential NaN for avg_ndwi_score (e.g., if no pixels intersected)
    # Treat NaN NDWI as neutral (0) for scoring, which normalizes to 0.5
    avg_ndwi = row['avg_ndwi_score'] if pd.notna(row['avg_ndwi_score']) else 0.0
    speed_knots = row['speed_knots']

    # --- NDWI Component (higher is better) ---
    # NDWI ranges from -1 to 1. Normalize to 0-1 range: (NDWI + 1) / 2
    # Then scale to a 0-50 point range (assuming equal weight with speed)
    ndwi_component = ((avg_ndwi + 1) / 2) * 50

    # --- Speed Component (lower is better) ---
    # Define a maximum speed for normalization. Speeds above this get 0 points for this component.
    max_speed_for_scoring = 6.0 # knots (e.g., typical speed limit in eco-sensitive zones)

    # Normalize inverse speed to 0-1 range: (max_speed - current_speed) / max_speed
    # Clip to 0-1 to ensure non-negative points for speed component
    inverse_speed_normalized = np.clip((max_speed_for_scoring - speed_knots) / max_speed_for_scoring, 0, 1)
    # Scale to a 0-50 point range
    speed_component = inverse_speed_normalized * 50

    # Combine components for the final eco-score (range 0-100)
    eco_score = ndwi_component + speed_component

    # Ensure the final score is within 0-100
    return np.clip(eco_score, 0, 100)

# Apply the scoring function to the GeoDataFrame
print("Calculating eco_score_ndwi_speed...")
gdf_vessels_ndwi_scored['eco_score_ndwi_speed'] = gdf_vessels_ndwi_scored.apply(calculate_eco_score_ndwi_speed, axis=1)

print("gdf_vessels_ndwi_scored with new 'eco_score_ndwi_speed' column (head):")
display(gdf_vessels_ndwi_scored.head())

Calculating eco_score_ndwi_speed...
gdf_vessels_ndwi_scored with new 'eco_score_ndwi_speed' column (head):


,id,vessel_type,speed_knots,timestamp,geometry,avg_ndwi_score,eco_score_ndwi_speed
0,1,research_vessel_high_NDWI,3.0,2025-11-08 10:00:21.394,"LINESTRING (13.25385 52.571, 13.25007 52.56782...",0.504131,62.603273
1,2,eco_tour_medium_NDWI,2.5,2025-11-08 16:00:21.395,"LINESTRING (13.28511 52.56726, 13.25254 52.578...",0.284545,61.280285


Apply the developed scoring logic to all vessels. Create a new GeoDataFrame containing the original vessel information along with their calculated eco-scores. Save this scored GeoDataFrame to a new GeoJSON file

In [ ]:
output_filename_scored = 'scored_vessels_ndwi.geojson'

try:
    gdf_vessels_ndwi_scored.to_file(output_filename_scored, driver="GeoJSON")
    print(f"\nSaved scored vessel routes to '{output_filename_scored}'.")
except ValueError as e:
    print(f"CRS save error ({e}), attempting to save with CRS=None workaround.")
    # Convert timestamp back to string for saving if there was an issue
    scored_routes_for_save = gdf_vessels_ndwi_scored.copy()
    if 'timestamp' in scored_routes_for_save.columns:
        scored_routes_for_save['timestamp'] = scored_routes_for_save['timestamp'].astype(str)

    scored_routes_no_crs = gpd.GeoDataFrame(scored_routes_for_save.drop(columns=['geometry']), geometry=scored_routes_for_save.geometry, crs=None)
    scored_routes_no_crs.to_file(output_filename_scored, driver="GeoJSON")
    print(f"\nSaved scored vessel routes to '{output_filename_scored}' using CRS=None workaround.")

print("\n--- Final Scored Vessels Data ---")
display(gdf_vessels_ndwi_scored[['id', 'vessel_type', 'avg_ndwi_score', 'speed_knots', 'eco_score_ndwi_speed']].head())


Saved scored vessel routes to 'scored_vessels_ndwi.geojson'.

--- Final Scored Vessels Data ---


,id,vessel_type,avg_ndwi_score,speed_knots,eco_score_ndwi_speed
0,1,research_vessel_high_NDWI,0.504131,3.0,62.603273
1,2,eco_tour_medium_NDWI,0.284545,2.5,61.280285
